# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものです。

## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [116]:
import pandas as pd
import numpy as np

## データの読み込み

In [128]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [118]:
df_receipt.loc[:,['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].query('customer_id == "CS018205000001" & (1000 <= amount | 5 <= quantity)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [119]:
df_customer.sort_values(by = 'birth_day', ascending = False).head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [120]:
df_receipt['customer_id'].count()

104681

## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [121]:
df_receipt['customer_id'].nunique()

8307

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [122]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [123]:
per_customer_id = df_receipt_only_member.loc[:,['customer_id','amount']].groupby('customer_id').sum()
average = per_customer_id['amount'].mean()
per_customer_id.query('@average <= amount').head(5)

,amount
customer_id,
CS001115000010,3044
CS001205000006,3337
CS001214000009,4685
CS001214000017,4132
CS001214000052,5639


---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [124]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [125]:
df_receipts = pd.merge(df_customer_only_member, df_receipt_only_member)
df_receipts.query('gender_cd == 1').loc[:,['customer_id', 'amount']].groupby('customer_id').sum()

,amount
customer_id,
CS001113000004,1298
CS001114000005,626
CS001115000010,3044
CS001211000025,456
CS001212000027,448
...,...
CS051212000001,336
CS051513000004,551
CS051515000002,265


---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [131]:
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format = '%Y%m%d')
df_customer.loc[:,['customer_id', 'application_date']].head(5)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05


---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [137]:
per_customer_id = df_receipt_only_member.loc[:,['customer_id','amount']].groupby('customer_id').sum()
per_customer_id = per_customer_id.assign(log_amount = np.log10(per_customer_id['amount']))
per_customer_id.loc[:,['amount','log_amount']].head(5)

,amount,log_amount
customer_id,,
CS001113000004,1298,3.113275
CS001114000005,626,2.796574
CS001115000010,3044,3.483445
CS001205000004,1988,3.298416
CS001205000006,3337,3.523356


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [138]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [139]:
df_product_1 = df_product.dropna(how = 'any')

In [140]:
len(df_product), len(df_product_1)

(10030, 10023)

---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [141]:
df_product_2 = df_product.fillna(df_product.mean().round())
df_product_2.isnull().sum()

<ipython-input-141-3edf280ada4f>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_product_2 = df_product.fillna(df_product.mean().round())


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64